In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from tqdm import tqdm

/kaggle/input/15th-data60sec/train_label.csv
/kaggle/input/15th-data60sec/train_data.pickle/train_data.pickle
/kaggle/input/15th-data60sec/test_data.pickle/test_data.pickle


In [2]:
train = pd.read_pickle('/kaggle/input/15th-data60sec/train_data.pickle/train_data.pickle')
test = pd.read_pickle('/kaggle/input/15th-data60sec/test_data.pickle/test_data.pickle')

- 문자열 포함된 id => 실제 데이터 => event time 10초
- 나머지 => 모사 데이터 => event time 0~15초

> 그렇다면 문자열 포함된 id는 10초 이전 버리고, 나머지는 0~15 까지 하나씩 시도할 수도 있을 듯.
> 그리고 문자열 포함된 열은 버리는 거로 하자

#### object feature 버리기

In [3]:
train = train.select_dtypes(exclude = 'object')
test = train.select_dtypes(exclude = 'object')
label = train['label']

#### duplicated 변수 찾아서 제외

In [4]:
var_list = list(set(list(train.mean().drop_duplicates().index)) & set(list(test.mean().drop_duplicates().index)))

#### constant 변수 찾아서 제외

In [5]:
constant_var = []
for col in tqdm(var_list):
    if train[col].nunique() == 1:
        constant_var += [col]

for col in tqdm(var_list):
    if test[col].nunique() == 1:
        constant_var += [col]
        
constant_var = list(np.unique(constant_var))

100%|██████████| 2618/2618 [00:06<00:00, 430.12it/s]


In [6]:
var_list = list(reversed(sorted(list(set(var_list) - set(constant_var)))))

In [7]:
train[var_list]

,time,label,id,V5120,V5118,V5117,V5116,V5090,V5089,V5088,...,V0009,V0008,V0007,V0006,V0005,V0004,V0003,V0002,V0001,V0000
0,0,999,0,-0.004474,-0.264703,112.263,127.58,43.212031,-0.125627,-0.237285,...,0.001467,0.0,-1.213967e-19,201.196818,204.576461,8.711798,8.691251,8.736399,8.682230,30.486806
1,1,999,0,0.000000,0.000014,0.000,0.00,43.200596,-0.165889,-0.222815,...,0.001899,0.0,-2.279681e-19,169.979333,194.222395,8.716143,8.706254,8.691244,8.784153,30.460030
2,2,999,0,0.000000,-0.000013,0.000,0.00,43.192798,-0.139667,-0.216915,...,0.001837,0.0,-1.586760e-19,160.719360,193.078139,8.681361,8.721917,8.697733,8.812022,30.475103
3,3,999,0,0.000000,0.000019,0.000,0.00,43.200013,-0.172449,-0.188965,...,-0.000897,0.0,-4.016423e-19,232.675873,207.644178,8.724655,8.723829,8.712004,8.908323,30.455413
4,4,999,0,0.000000,-0.000008,0.000,0.00,43.199240,-0.169157,-0.181302,...,0.001726,0.0,8.676680e-19,160.829340,186.112451,8.702341,8.722939,8.702882,8.815620,30.472642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49675,55,156,99,0.000000,-0.000020,0.000,0.00,43.198699,-0.174953,-0.170195,...,0.001896,0.0,6.307583e-19,212.384571,209.257888,8.714446,8.666724,8.689265,8.680374,30.476523
49676,56,156,99,0.000000,-0.000009,0.000,0.00,43.200190,-0.152945,-0.151160,...,0.000458,0.0,-2.693599e-19,260.248417,233.122683,8.719074,8.699017,8.688116,8.818355,30.464995
49677,57,156,99,0.000000,0.000001,0.000,0.00,43.199581,-0.163286,-0.218112,...,0.000130,0.0,3.863359e-19,201.468825,260.322342,8.656740,8.712057,8.706527,8.737560,30.483490
49678,58,156,99,0.000000,0.000015,0.000,0.00,43.197625,-0.165087,-0.196520,...,-0.001420,0.0,2.594330e-19,242.729158,232.156914,8.698854,8.719814,8.685300,8.696491,30.459609


In [8]:
test[var_list]

,time,label,id,V5120,V5118,V5117,V5116,V5090,V5089,V5088,...,V0009,V0008,V0007,V0006,V0005,V0004,V0003,V0002,V0001,V0000
0,0,999,0,-0.004474,-0.264703,112.263,127.58,43.212031,-0.125627,-0.237285,...,0.001467,0.0,-1.213967e-19,201.196818,204.576461,8.711798,8.691251,8.736399,8.682230,30.486806
1,1,999,0,0.000000,0.000014,0.000,0.00,43.200596,-0.165889,-0.222815,...,0.001899,0.0,-2.279681e-19,169.979333,194.222395,8.716143,8.706254,8.691244,8.784153,30.460030
2,2,999,0,0.000000,-0.000013,0.000,0.00,43.192798,-0.139667,-0.216915,...,0.001837,0.0,-1.586760e-19,160.719360,193.078139,8.681361,8.721917,8.697733,8.812022,30.475103
3,3,999,0,0.000000,0.000019,0.000,0.00,43.200013,-0.172449,-0.188965,...,-0.000897,0.0,-4.016423e-19,232.675873,207.644178,8.724655,8.723829,8.712004,8.908323,30.455413
4,4,999,0,0.000000,-0.000008,0.000,0.00,43.199240,-0.169157,-0.181302,...,0.001726,0.0,8.676680e-19,160.829340,186.112451,8.702341,8.722939,8.702882,8.815620,30.472642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49675,55,156,99,0.000000,-0.000020,0.000,0.00,43.198699,-0.174953,-0.170195,...,0.001896,0.0,6.307583e-19,212.384571,209.257888,8.714446,8.666724,8.689265,8.680374,30.476523
49676,56,156,99,0.000000,-0.000009,0.000,0.00,43.200190,-0.152945,-0.151160,...,0.000458,0.0,-2.693599e-19,260.248417,233.122683,8.719074,8.699017,8.688116,8.818355,30.464995
49677,57,156,99,0.000000,0.000001,0.000,0.00,43.199581,-0.163286,-0.218112,...,0.000130,0.0,3.863359e-19,201.468825,260.322342,8.656740,8.712057,8.706527,8.737560,30.483490
49678,58,156,99,0.000000,0.000015,0.000,0.00,43.197625,-0.165087,-0.196520,...,-0.001420,0.0,2.594330e-19,242.729158,232.156914,8.698854,8.719814,8.685300,8.696491,30.459609
